In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

import datetime, warnings, scipy 
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime, warnings, scipy 
#from mpl_toolkits.basemap import Basemap

import warnings
warnings.filterwarnings(action='once')
from geopandas import points_from_xy
import geopandas as gdp

/var/folders/ps/l253_yxx53sdg2mn5d6_gmgr0000gn/T/ipykernel_2816/94487599.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


### Step 1 - Cleaning

In [ ]:
# Here we build the function to automotize the cleaning of the data sets
def Cleaning(data): 
    
    #removing empty columns
    Variables_to_remove = ['lot1_numero', 'lot1_surface_carrez', 
                       'lot2_numero', 'lot2_surface_carrez',
                       'lot3_numero', 'lot3_surface_carrez', 
                       'lot4_numero', 'lot4_surface_carrez', 
                       'lot5_numero', 'lot5_surface_carrez',
                       'code_nature_culture_speciale',  
                       'nature_culture_speciale',
                       'ancien_id_parcelle',
                       'adresse_suffixe', 'ancien_code_commune','ancien_nom_commune', 
                       'id_parcelle',
                       'numero_volume', 
                       'type_local', 'adresse_code_voie',   #redundant variables
                       'code_nature_culture',
                       'code_postal'] # on va utiliser le code commune comme il est replis a 100%

    data = data.drop(Variables_to_remove, axis=1)
    
    #removing empty rows
    data = data[(data.valeur_fonciere.isna() == False)   
               & (data.latitude.isna() == False) 
               & (data.longitude.isna() == False)
               & (data['nature_mutation'] == "Vente") 
               ]
    
    #to keep the row if nature_culture is empty but code-type_local is filled
    data = data[(data.nature_culture.isna()) | (data.code_type_local.isna() == False)]
    
    #we limited our self for the appartments, houses and dependences. so we exclude 'loal commerciales'
    data = data[(data['code_type_local'] == 1.0)        #Maison
               | (data['code_type_local'] == 2.0)       #appartement
               | (data['code_type_local'] == 3.0)]      # dependance
    
    
    data = data[(data['nature_culture'] != "terrains a bâtir")]
    

    #fixing the problem of multiple Id_mutation for the same sale. 
    #we have for the same sale a dependence and a house/appartment so we aggregated them in one row with on id_mutation.
    
    #in This fucntion we used only the first proposition(waiting to the meeting)
    data = data[(data['nature_culture'] == "sols") 
                | (data.nature_culture.isna()) ].groupby(['id_mutation','numero_disposition','date_mutation'],
                                                    as_index=False).agg(
                                                       {'date_mutation':'first',
                                                        'code_type_local': 'min', 
                                                        'code_commune':'max',
                                                        'surface_terrain': 'max', 
                                                        'surface_reelle_bati':'sum',
                                                        'nombre_pieces_principales':'sum',
                                                        'nature_culture': 'sum',
                                                        'valeur_fonciere':'max', 
                                                        'latitude':'max' , 'longitude':'max',
                                                        'nombre_lots':'max',
                                                        'numero_disposition': 'max',
                                                        'code_departement':'first'
                                                        })
    
    #setting the data for appartments and houses
    data = data[(data.code_type_local.isna() == False)]
    
    data = data[(data['code_type_local'] == 1.0)        #Maison
               | (data['code_type_local'] == 2.0) ]     #appartement   
    
    data['surface_reelle_bati'].fillna(0, inplace=True)
    data['nombre_pieces_principales'].fillna(0, inplace=True)
    data['surface_terrain'].fillna(0, inplace = True)
    
    data['date_mutation'] = pd.to_datetime(data['date_mutation'], format="%Y-%m-%d")

    # NOMBRE DE PIECES
    # On exclut les biens dont le nombre de pièces est inférieure à 1 et supérieur à 20 :
    
    data.drop(data[data['nombre_pieces_principales']<1].index, inplace=True)
    data.drop(data[data['nombre_pieces_principales']>20].index, inplace=True)

    # SURFACE DU BIEN 
    # On exclut les biens dont la surface est inférieure au seuil légal de 9 mètres carrés pour la location :
    data.drop(data[data['surface_reelle_bati']<10].index, inplace=True)
    data.drop(data[data['valeur_fonciere'] < 2000].index, inplace=True)

    
    return data


In [ ]:
def Cleaning_iris(df_iris):
    Variables_to_remove = ['ze2010_code', 'ze2010_name', 'ept_name', 
                            'ept_code', 'ze2020_name', 'ze2020_code',
                            'arrdep_name']

    df_iris = df_iris.drop(Variables_to_remove, axis=1)
    
    #removing empty rows
    df_iris = df_iris[(df_iris.geometry.isna() == False) & (df_iris.epci_name.isna() == False)]
    
    return df_iris

### Step 2 - Adding Spatial index (IRIS)

In [ ]:
def Add_IRIS(data, df_iris):    
    # turn df_test into geodataframe
    df_spatial = gdp.GeoDataFrame(data, crs="EPSG:4326", 
                                  geometry=points_from_xy(
                                     data["longitude"], data["latitude"]),)
    data_spatial = gdp.sjoin(df_spatial, df_iris,how='left', 
                               predicate="within",)
    
    return data_spatial

def plot_map(data_spatial):
    
    return data_spatial[data_spatial['Year'] == 2017].drop(['date_mutation'],axis = 1).explore(tiles="CartoDB positron",
                                cmap="tab20b", column="iris_code")

### Step 3 - Processing

In [ ]:
def Prix_m2(data):
    data['Prix_m2'] = data['valeur_fonciere'] / data['surface_reelle_bati']
    data['Year'] = pd.DatetimeIndex(data['date_mutation']).year
    data['Month'] = pd.DatetimeIndex(data['date_mutation']).month
    
    #Somme du nombre de mètre carré de surface habitable vendue par commune
    Sum_m2_bati_par_iris_par_an = data.groupby(['iris_code','Year'])['surface_reelle_bati'].sum().reset_index()
    Sum_m2_bati_par_iris_par_an.rename(columns={'surface_reelle_bati': 'Sum_surface_bati_iris'}, inplace=True)
    data = data.merge(Sum_m2_bati_par_iris_par_an, on=['iris_code','Year'])
    
    #Somme du nombre de mètre carré de surface terrain vendue par commune
    Sum_m2_terr_par_iris_par_an = data.groupby(['iris_code','Year'])['surface_terrain'].sum().reset_index()
    Sum_m2_terr_par_iris_par_an.rename(columns={'surface_terrain': 'Sum_surface_terr_iris'}, inplace=True)
    data = data.merge(Sum_m2_terr_par_iris_par_an, on=['iris_code','Year'])

    #Somme des ventes par commune
    Sum_ventes_par_iris_par_an = data.groupby(['iris_code','Year'])['valeur_fonciere'].sum().reset_index()
    Sum_ventes_par_iris_par_an.rename(columns={'valeur_fonciere': 'Sum_valeur_fonc'}, inplace=True)
    data = data.merge(Sum_ventes_par_iris_par_an, on=['iris_code','Year'])

    #Prix moyen par commune du mètre carré de surface habitable
    data = data.assign(prix_m2_moy_surf_habit=(data["Sum_valeur_fonc"]
                                                    /data["Sum_surface_bati_iris"]))
    
    data = data.assign(prix_m2_moy_terrain=(data["Sum_valeur_fonc"]
                                                    /data["Sum_surface_terr_iris"]))

    #Suppression des variables temporaires 
    data=data.drop(['Sum_surface_bati_iris', 'Sum_surface_terr_iris' , 'Sum_valeur_fonc'],axis=1)
    data.replace([np.inf, -np.inf], 0, inplace=True)
    
    return data

In [ ]:
def Process_data(data, quantile_low, quantile_high):    
    
    #we remove the extreme 5 % values of price per meter_square
    Threshold_up = data.groupby(['iris_code','Year'])['Prix_m2'].quantile(quantile_high).reset_index()
    Threshold_down = data.groupby(['iris_code','Year'])['Prix_m2'].quantile(quantile_low).reset_index()

    df_comune ={'iris_code':Threshold_up['iris_code'], 'Threshold_up_iris': Threshold_up['Prix_m2'] , 
            'Threshold_down_iris':Threshold_down['Prix_m2'] , 'Year': Threshold_down['Year']}
    df_comune = pd.DataFrame(df_comune)

    df_process = pd.merge(data, df_comune ,how='inner', on=['iris_code','Year'])

    df_final = df_process[(df_process['Prix_m2'] < df_process['Threshold_up_iris']) 
                      & (df_process['Prix_m2'] > df_process['Threshold_down_iris'])]
    
    df_final=df_final.drop(['Threshold_up_iris','Threshold_down_iris'],axis = 1)
    
    return df_final


### Running the Programme

#### Loading Data

In [ ]:
#loading dvf data sets
df17 = pd.read_csv('./datadvf2017.csv',low_memory=False)
df18 = pd.read_csv('./datadvf2018.csv',low_memory=False)
df19 = pd.read_csv('./datadvf2019.csv',low_memory=False)
df20 = pd.read_csv('./datadvf2020.csv',low_memory=False)
df21 = pd.read_csv('./datadvf2021.csv',low_memory=False)
df22 = pd.read_csv('./datadvf2022.csv',low_memory=False)

#loading IRIS datasets
df_iris_2022 = gdp.read_file( "./georef-france-iris-millesime.shp")

#### Applying Step 1 and Concating in one dataframe

In [ ]:
df_cleaned17 = Cleaning(df17)
df_cleaned18 = Cleaning(df18)
df_cleaned19 = Cleaning(df19)
df_cleaned20 = Cleaning(df20)
df_cleaned21 = Cleaning(df21)
df_cleaned22 = Cleaning(df22)

df_iris = Cleaning_iris(df_iris_2022)

# Concat all data
df_cleaned_tot = pd.concat([df_cleaned17, df_cleaned18, 
                            df_cleaned19, df_cleaned20, 
                            df_cleaned21, df_cleaned22] , axis=0 , ignore_index=True)

### Paris

In [ ]:
df_Paris = df_cleaned_tot[df_cleaned_tot['code_departement'] == '75']

#### Applying Step 2 - Adding spatial features

In [ ]:
df_Paris_spatial = Add_IRIS(df_Paris, df_iris)

#### Applying Step 3 - Processing data

In [ ]:
df_Paris_price = Prix_m2(df_Paris_spatial)
df_Paris_Processed = Process_data(df_Paris_price, 0.05, 0.95)

In [ ]:
plot_map(df_Paris_Processed)

### Marseille

In [ ]:
df_Marseille = df_cleaned_tot[(df_cleaned_tot['code_commune'] == '13201') 
                              | (df_cleaned_tot['code_commune'] == '13202')
                              | (df_cleaned_tot['code_commune'] == '13203')
                              | (df_cleaned_tot['code_commune'] == '13204')
                              | (df_cleaned_tot['code_commune'] == '13205')
                              | (df_cleaned_tot['code_commune'] == '13206')
                              | (df_cleaned_tot['code_commune'] == '13207')
                              | (df_cleaned_tot['code_commune'] == '13208')
                              | (df_cleaned_tot['code_commune'] == '13209')
                              | (df_cleaned_tot['code_commune'] == '13210')
                              | (df_cleaned_tot['code_commune'] == '13211')
                              | (df_cleaned_tot['code_commune'] == '13212')
                              | (df_cleaned_tot['code_commune'] == '13213')
                              | (df_cleaned_tot['code_commune'] == '13214')
                              | (df_cleaned_tot['code_commune'] == '13215')
                              | (df_cleaned_tot['code_commune'] == '13216')]

In [ ]:
df_Marseille_spatial = Add_IRIS(df_Marseille, df_iris)

In [ ]:
df_Marseille_price = Prix_m2(df_Marseille_spatial)
df_Marseille_Processed = Process_data(df_Marseille_price, 0.05, 0.95)

In [ ]:
plot_map(df_Marseille_Processed)

### Lyon

In [ ]:
df_Lyon = df_cleaned_tot[(df_cleaned_tot['code_commune'] == '69381') 
                              | (df_cleaned_tot['code_commune'] == '69382')
                              | (df_cleaned_tot['code_commune'] == '69383')
                              | (df_cleaned_tot['code_commune'] == '69384')
                              | (df_cleaned_tot['code_commune'] == '69385')
                              | (df_cleaned_tot['code_commune'] == '69386')
                              | (df_cleaned_tot['code_commune'] == '69387')
                              | (df_cleaned_tot['code_commune'] == '69388')
                              | (df_cleaned_tot['code_commune'] == '69389')]

#### Applying Step 2 - Adding spatial features

In [ ]:
df_Lyon_spatial = Add_IRIS(df_Lyon, df_iris)

#### Applying Step 3 - Processing data

In [ ]:
df_Lyon_price = Prix_m2(df_Lyon_spatial)
df_Lyon_Processed = Process_data(df_Lyon_price, 0.05, 0.95)

### Map - Lyon

In [ ]:
plot_map(df_Lyon_Processed)

### Toulouse

In [ ]:
df_Toulouse = df_cleaned_tot[(df_cleaned_tot['code_commune'] == '31555')]

#### Applying Step 2 - Adding spatial features

In [ ]:
df_Toulouse_spatial = Add_IRIS(df_Toulouse, df_iris)

#### Applying Step 3 - Processing data

In [ ]:
df_Toulouse_price = Prix_m2(df_Toulouse_spatial)
df_Toulouse_Processed = Process_data(df_Toulouse_price, 0.05, 0.95)

### Map - Toulouse

In [ ]:
plot_map(df_Toulouse_Processed)

### Function Compiling all the Procedure

In [ ]:
def Procedure(data_cleaned):
    
    data_cleaned_spatial = Add_IRIS(data_cleaned, df_iris)
    df_cleaned_price = Prix_m2(data_cleaned_spatial)
    data_cleaned_Processed = Process_data(df_cleaned_price, 0.05, 0.95)
    
    return plot_map(data_cleaned_Processed)

We need to just apply the first step 'the cleaning function then concat the data', once we have that we apply directly the Procedure function on the data for example for Paris: 

In [ ]:
def Procedure(data_cleaned):
    
    data_cleaned_spatial = Add_IRIS(data_cleaned, df_iris)
    df_cleaned_price = Prix_m2(data_cleaned_spatial)
    data_cleaned_Processed = Process_data(df_cleaned_price, 0.05, 0.95)
    
    return plot_map(data_cleaned_Processed)

In [ ]:
Procedure(df_Paris)

### Addition of the average price evolution by IRIS

In [261]:
df_Paris_Processed = pd.read_csv('/Users/admin/Desktop/BDC-Dev-Antonin/Data/df_Paris_Processed.csv',low_memory=False)

In [266]:
def Average_price_by_IRIS(df_Paris_Processed):

    df_Paris_Processed['date_mutation'] = pd.to_datetime(df_Paris_Processed['date_mutation'], format="%Y-%m-%d")

    #Création de la variable Trimestre
    df_Paris_Processed['trimestre_simple'] = df_Paris_Processed['date_mutation'].dt.quarter
    df_Paris_Processed.loc[df_Paris_Processed['trimestre_simple'] == 1, 'trimestre_simple'] = '01-01'
    df_Paris_Processed.loc[df_Paris_Processed['trimestre_simple'] == 2, 'trimestre_simple'] = '04-01'
    df_Paris_Processed.loc[df_Paris_Processed['trimestre_simple'] == 3, 'trimestre_simple'] = '07-01'
    df_Paris_Processed.loc[df_Paris_Processed['trimestre_simple'] == 4, 'trimestre_simple'] = '10-01'


    #Création de la variable Trimestre par année
    df_Paris_Processed['trimestre'] = df_Paris_Processed['Year'].astype(str).str.cat(df_Paris_Processed['trimestre_simple'].astype(str), sep='-')
    
    #Somme du nombre de mètre carré de surface habitable vendue par trimestre par IRIS
    Sum_m2_bati_par_iris = df_Paris_Processed.groupby(['trimestre','iris_name_l'])['surface_reelle_bati'].sum().reset_index()
    Sum_m2_bati_par_iris.rename(columns={'surface_reelle_bati': 'Sum_surface_bati_iris'}, inplace=True)
    df_Paris_Processed=df_Paris_Processed.merge(Sum_m2_bati_par_iris, on=['trimestre', 'iris_name_l'])

    #Somme des ventes par trimestre par IRIS
    Sum_ventes_par_iris = df_Paris_Processed.groupby(['trimestre','iris_name_l'])['valeur_fonciere'].sum().reset_index()
    Sum_ventes_par_iris.rename(columns={'valeur_fonciere': 'Sum_valeur_fonc'}, inplace=True)
    df_Paris_Processed=df_Paris_Processed.merge(Sum_ventes_par_iris, on=['trimestre', 'iris_name_l'])

    #Prix moyen par IRIS du mètre carré de surface habitable
    df_Paris_Processed=df_Paris_Processed.assign(prix_m2_moy_IRIS_trimestri_surf_habit=(df_Paris_Processed["Sum_valeur_fonc"]
                                                        /df_Paris_Processed["Sum_surface_bati_iris"]))

    #Suppression des variables temporaires 
    df_Paris_Processed = df_Paris_Processed.drop(['Sum_surface_bati_iris', 'Sum_valeur_fonc', 'trimestre_simple'],axis=1)
    
    #Les observations pour lesquelles les valeurs de 'trimestre' pour chaque valeur 
    #de 'iris_name_l' ne contiennent pas toutes les valeurs possibles dans 'trimestre' sont supprimées du dataframe.
    counts = df_Paris_Processed.groupby('iris_name_l')['trimestre'].transform('nunique')
    df_Paris_Processed = df_Paris_Processed[counts == df_Paris_Processed['trimestre'].nunique()]
    #Perte de 5000 Observations

    # CALCUL DU TAUX D'ACCROISSEMENT 
    df_prix_m2_moy_IRIS = df_Paris_Processed.loc[:, ["iris_name_l", "trimestre", "prix_m2_moy_IRIS_trimestri_surf_habit"]]
    # Sélection des valeurs uniques en utilisant la méthode drop_duplicates()
    df_prix_m2_moy_IRIS = df_prix_m2_moy_IRIS.drop_duplicates()
    df_prix_m2_moy_IRIS = df_prix_m2_moy_IRIS.sort_values(["iris_name_l", "trimestre"])
    # Dataframe groupé par IRIS
    grouped_df = df_prix_m2_moy_IRIS.groupby("iris_name_l")
    df_prix_m2_moy_IRIS["taux_accroissement"] = grouped_df["prix_m2_moy_IRIS_trimestri_surf_habit"].pct_change()
    df_prix_m2_moy_IRIS["taux_accroissement"] = df_prix_m2_moy_IRIS["taux_accroissement"].fillna(0)
    df_Paris_Processed=df_Paris_Processed.merge(df_prix_m2_moy_IRIS, on=['iris_name_l','trimestre','prix_m2_moy_IRIS_trimestri_surf_habit'])
    
    return df_Paris_Processed

In [268]:
df_Paris_Processed_with_average_price = Average_price_by_IRIS(df_Paris_Processed)

In [269]:
df_Paris_Processed_with_average_price

,id_mutation,date_mutation,code_type_local,code_commune,surface_terrain,surface_reelle_bati,nombre_pieces_principales,nature_culture,valeur_fonciere,latitude,...,iris_grd_qu,iris_in_ctu,Prix_m2,Year,Month,prix_m2_moy_surf_habit,prix_m2_moy_terrain,trimestre,prix_m2_moy_IRIS_trimestri_surf_habit,taux_accroissement
0,2017-731653,2017-07-04,2.0,75103,0.0,42.0,3.0,0,571110.0,48.858043,...,Les Archives,Non,13597.857143,2017,7,10436.251897,0.0,2017-07-01,12279.057239,0.000000
1,2017-731668,2017-07-07,2.0,75103,0.0,88.0,3.0,0,1414600.0,48.857626,...,Les Archives,Non,16075.000000,2017,7,10436.251897,0.0,2017-07-01,12279.057239,0.000000
2,2017-731783,2017-07-19,2.0,75103,0.0,17.0,1.0,0,215000.0,48.856136,...,Les Archives,Non,12647.058824,2017,7,10436.251897,0.0,2017-07-01,12279.057239,0.000000
3,2017-731808,2017-07-27,2.0,75103,0.0,72.0,3.0,0,598000.0,48.857626,...,Les Archives,Non,8305.555556,2017,7,10436.251897,0.0,2017-07-01,12279.057239,0.000000
4,2017-731873,2017-07-28,2.0,75103,0.0,32.0,2.0,0,360000.0,48.856914,...,Les Archives,Non,11250.000000,2017,7,10436.251897,0.0,2017-07-01,12279.057239,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139505,2022-530177,2022-03-29,2.0,75113,0.0,35.0,1.0,0,12000.0,48.821917,...,Maison Blanche,Non,342.857143,2022,3,5330.979499,0.0,2022-01-01,4604.166667,-0.386695
139506,2022-531147,2022-04-08,2.0,75113,0.0,47.0,2.0,0,365000.0,48.821172,...,Maison Blanche,Non,7765.957447,2022,4,5330.979499,0.0,2022-04-01,5359.219858,0.163993
139507,2022-536119,2022-06-02,2.0,75113,0.0,60.0,2.0,0,476000.0,48.821917,...,Maison Blanche,Non,7933.333333,2022,6,5330.979499,0.0,2022-04-01,5359.219858,0.163993
139508,2022-536803,2022-06-30,2.0,75113,0.0,58.0,2.0,0,380000.0,48.821917,...,Maison Blanche,Non,6551.724138,2022,6,5330.979499,0.0,2022-04-01,5359.219858,0.163993


In [270]:
# Exportation de la nouvelle base
df_Paris_Processed_with_average_price.to_csv(r'/Users/admin/Desktop/BDC-Dev-Antonin/Data/df_Paris_Processed_with_av_price.csv', index=False)